# Tear Down

We start by deleting the current cluster

In [ ]:
!gcloud container --project "seldon-demo" clusters delete seldon-demo-cluster --quiet

Then we delete the current forwarding rules

In [ ]:
import os
result = os.popen("gcloud compute forwarding-rules list").read()
forwarding_rules = [x.split(' ')[0] for x in result.split('\n')[1:-1]]
for fr in forwarding_rules:
    print 'Deleting rule {}...'.format(fr)
    os.system("gcloud compute forwarding-rules delete {} --quiet".format(fr))
    print 'Deleted.'

Create a new cluster

In [ ]:
!gcloud container --project "seldon-demo" clusters create "seldon-demo-cluster" \
    --zone "europe-west1-b" --machine-type "n1-standard-4" \
    --scopes "https://www.googleapis.com/auth/compute","https://www.googleapis.com/auth/devstorage.read_only","https://www.googleapis.com/auth/logging.write","https://www.googleapis.com/auth/servicecontrol","https://www.googleapis.com/auth/service.management.readonly" \
    --num-nodes "1" --network "default" --enable-cloud-logging --no-enable-cloud-monitoring \
    --image-type=container_vm


# Set Up

We create the weave pods on the cluster (probe and app)

In [ ]:
!kubectl create -f 'https://scope.weave.works/launch/k8s/weavescope.yaml' --validate=false

We expose the weave app pod externally

In [ ]:
%%bash
pod=$(kubectl get pod --selector=weavescope-component=weavescope-app -o jsonpath={.items..metadata.name})
kubectl expose pod $pod --type="LoadBalancer" --port=4040 --target-port=4040 

The Weave scope app will be available on the ip address and port listed under EXTERNAL-IP and PORT(S) in the table below

In [ ]:
%%bash
pod=$(kubectl get pod --selector=weavescope-component=weavescope-app -o jsonpath={.items..metadata.name})
kubectl get services $pod

We start seldon on the new cluster (takes 5+ minutes)

In [ ]:
!/home/ubuntu/git/seldon-server/kubernetes/bin/seldon-up.sh

In [ ]:
!kubectl get services